## Schools Filtering Fom Starbucks filter

In [16]:
import pandas as pd
import re
import getpass
import json
import numpy as np
import json
from IPython.display import clear_output
import getLocation as gl
import googleNearBy_API as near

In [17]:
with open('./output/merge_stb_1M.json', 'r') as myfile:
    data=myfile.read()
obj = json.loads(data)
stb_1M=pd.DataFrame(obj)
stb_1M.head()

,ref,Latitude_starbucks,Longitude_starbucks,geoloc_stb,Name,Category,Description,Latitude,Longitude,Founded Year,Total Money Raised,Raised Amount,Currency,Address,Zip code,City,GeoLocations
0,5dd59811fc1c02fda798c75c,37.790070,-122.402083,"{'type': 'Point', 'coordinates': [-122.4020826...",Scribd,news,Read Unlimited Books,37.789634,-122.404052,2007.0,$25.8M,25762000.0,USD,539 Bryant Street,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.404052,..."
1,5dd59811fc1c02fda798c75d,37.767287,-122.393926,"{'type': 'Point', 'coordinates': [-122.3939261...",Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,5dd59811fc1c02fda798c75e,37.446737,-122.160489,"{'type': 'Point', 'coordinates': [-122.1604894...",Lala,games_video,hub for online music discovery,37.451151,-122.154369,NaN,$44.2M,44150000.0,USD,209 Hamilton Ave,94301,Palo Alto,"{'type': 'Point', 'coordinates': [-122.154369,..."
3,5dd59811fc1c02fda798c75f,33.023418,-117.081877,"{'type': 'Point', 'coordinates': [-117.081877,...",Slacker,music,Music Personalization Suite,33.022176,-117.081406,2006.0,$73.1M,73100000.0,USD,16935 W. Bernardo Dr. Suite 101,92127,San Diego,"{'type': 'Point', 'coordinates': [-117.081406,..."
4,5dd59811fc1c02fda798c760,40.759393,-73.985288,"{'type': 'Point', 'coordinates': [-73.985288, ...",MeetMoi,social,Mobile Dating,40.757929,-73.985506,2007.0,$5.58M,5575000.0,USD,,,New York City,"{'type': 'Point', 'coordinates': [-73.985506, ..."


#### NOTA: Llamada a la google API 812 veces!

- Searching for schools in a radius of 1km from each office.

In [22]:
import time
count=0
schools=[]
for _,row in stb_1M.iterrows():
    clear_output()
    count+=1
    print(count)
    s=dict()
    lat=row['Latitude']
    long=row['Longitude']
    dat=near.placesearch(lat,long,1000,"Elementary School")
    if dat['status']!='ZERO_RESULTS' and dat['status']!='REQUEST_DENIED'and dat['status']!='INVALID_REQUEST':
        lt=dat['results'][0]['geometry']['location']['lat']
        lg=dat['results'][0]['geometry']['location']['lng']
        s['ref']=row['ref']
        s['latitude_schools']=lt
        s['longitude_schools']=lg
        s['geoloc_school']=gl.getLocation(lg,lt)
        schools.append(s) 
    time.sleep(1)

812


In [23]:
sch_stb_1M=pd.DataFrame(schools)

In [30]:
sch_stb_1M.head()

,ref,latitude_schools,longitude_schools,geoloc_school
0,5dd59811fc1c02fda798c75c,37.798463,-122.403093,"{'type': 'Point', 'coordinates': [-122.4030926..."
1,5dd59811fc1c02fda798c75d,37.760581,-122.395844,"{'type': 'Point', 'coordinates': [-122.3958437..."
2,5dd59811fc1c02fda798c75e,37.443089,-122.143181,"{'type': 'Point', 'coordinates': [-122.1431815..."
3,5dd59811fc1c02fda798c75f,33.030247,-117.086845,"{'type': 'Point', 'coordinates': [-117.0868455..."
4,5dd59811fc1c02fda798c760,40.765788,-73.989530,"{'type': 'Point', 'coordinates': [-73.98953019..."


- Merging to have companies with schools and starbucks near.

In [26]:
merge_sch_stb=pd.merge(sch_stb_1M,stb_1M,how="inner",on='ref')

In [25]:
stb_1M.shape

(812, 17)

In [28]:
merge_sch_stb.to_json('./output/StarbucksSchool_1M.json', orient="records")

In [31]:
merge_sch_stb.head()

,ref,latitude_schools,longitude_schools,geoloc_school,Latitude_starbucks,Longitude_starbucks,geoloc_stb,Name,Category,Description,Latitude,Longitude,Founded Year,Total Money Raised,Raised Amount,Currency,Address,Zip code,City,GeoLocations
0,5dd59811fc1c02fda798c75c,37.798463,-122.403093,"{'type': 'Point', 'coordinates': [-122.4030926...",37.790070,-122.402083,"{'type': 'Point', 'coordinates': [-122.4020826...",Scribd,news,Read Unlimited Books,37.789634,-122.404052,2007.0,$25.8M,25762000.0,USD,539 Bryant Street,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.404052,..."
1,5dd59811fc1c02fda798c75d,37.760581,-122.395844,"{'type': 'Point', 'coordinates': [-122.3958437...",37.767287,-122.393926,"{'type': 'Point', 'coordinates': [-122.3939261...",Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,5dd59811fc1c02fda798c75e,37.443089,-122.143181,"{'type': 'Point', 'coordinates': [-122.1431815...",37.446737,-122.160489,"{'type': 'Point', 'coordinates': [-122.1604894...",Lala,games_video,hub for online music discovery,37.451151,-122.154369,NaN,$44.2M,44150000.0,USD,209 Hamilton Ave,94301,Palo Alto,"{'type': 'Point', 'coordinates': [-122.154369,..."
3,5dd59811fc1c02fda798c75f,33.030247,-117.086845,"{'type': 'Point', 'coordinates': [-117.0868455...",33.023418,-117.081877,"{'type': 'Point', 'coordinates': [-117.081877,...",Slacker,music,Music Personalization Suite,33.022176,-117.081406,2006.0,$73.1M,73100000.0,USD,16935 W. Bernardo Dr. Suite 101,92127,San Diego,"{'type': 'Point', 'coordinates': [-117.081406,..."
4,5dd59811fc1c02fda798c760,40.765788,-73.989530,"{'type': 'Point', 'coordinates': [-73.98953019...",40.759393,-73.985288,"{'type': 'Point', 'coordinates': [-73.985288, ...",MeetMoi,social,Mobile Dating,40.757929,-73.985506,2007.0,$5.58M,5575000.0,USD,,,New York City,"{'type': 'Point', 'coordinates': [-73.985506, ..."
